In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, PolynomialFeatures
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def evalerror(y, y_pred):
    loss = np.sum(np.square(y - y_pred))
    n = len(y)
    return loss / n

In [3]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('train/test.csv')
y = pd.read_csv('train/y.csv')

In [4]:
#GBDT筛选特征
clf_gt2 = ensemble.GradientBoostingRegressor(max_depth=1, n_estimators=320, random_state=1)
clf_gt2.fit(train, y)

model1 = SelectFromModel(clf_gt2, prefit=True) 
train = pd.DataFrame(model1.transform(train))
test = pd.DataFrame(model1.transform(test))

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
#线上xgb单模型
param = {}
param['eta'] = 0.01
param['max_depth'] = 6
#param['mmin_child_weight'] = 5
param['subsample'] = 0.8
param['colsample_bytree'] = 0.3
num_round = 750

xgbTrain = xgb.DMatrix(train, label=y)
xgbTest = xgb.DMatrix(test)
modle = xgb.train(param, xgbTrain, num_round, )
result_xgb = modle.predict(xgbTest)

In [8]:
#线上Stacking模型
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, 
                                              max_depth=2, 
                                              subsample=0.8, 
                                              learning_rate=0.01, 
                                              random_state=0, 
                                              max_features=0.2)
modle0  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.2, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=2100)
modle1  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.3, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=1600,
                           min_child_weight=6)

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3,
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=350, 
                         seed=0)
base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['lgb', clf1],]

folds = list(KFold(len(train), n_folds=5, random_state=0))
S_train = np.zeros((train.shape[0], len(base_model)))
S_test = np.zeros((test.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((test.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = train.ix[train_idx, :]
        X_valid = train.ix[test_idx, :]
        Y = y.ix[train_idx, :]
        clf.fit(X_train, Y['Y'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(test)       
    S_test[:, index] = S_test_i.mean(1)
    
linreg = linear_model.LinearRegression()
linreg.fit(S_train, y)

result = linreg.predict(S_test)

the model 0
the model 1
the model 2
the model 3


In [10]:
sub = pd.read_csv('data/测试A-答案模板.csv', names=['ID'])
sub['res'] = pd.DataFrame(result)[0]

In [128]:
sub.to_csv('submission{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)